In [1]:
config={
  "version": "v1",
  "config": {
    "visState": {
      "filters": [
        {
          "dataId": [
            "trajectories"
          ],
          "id": "ci8tl8aip",
          "name": [
            "date_time"
          ],
          "type": "timeRange",
          "value": [
            1376524868000,
            1376747899999.9995
          ],
          "enlarged": True,
          "plotType": "histogram",
          "animationWindow": "free",
          "yAxis": None
        }
      ],
      "layers": [
        {
          "id": "w0gvb3l",
          "type": "geojson",
          "config": {
            "dataId": "trajectories",
            "label": "trajectories",
            "color": [
              136,
              87,
              44
            ],
            "columns": {
              "geojson": "geometry"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.4,
              "strokeOpacity": 0.8,
              "thickness": 0.5,
              "strokeColor": None,
              "colorRange": {
                "name": "ColorBrewer Dark2-3",
                "type": "qualitative",
                "category": "ColorBrewer",
                "colors": [
                  "#1b9e77",
                  "#d95f02",
                  "#7570b3"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 8,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": False,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "bird_name",
              "type": "string"
            },
            "colorScale": "ordinal",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": None,
            "radiusScale": "linear"
          }
        },
        {
          "id": "nsjcvzk",
          "type": "geojson",
          "config": {
            "dataId": "stops",
            "label": "stops",
            "color": [
              18,
              147,
              154
            ],
            "columns": {
              "geojson": "geometry"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.3,
              "strokeOpacity": 0.8,
              "thickness": 0.1,
              "strokeColor": [
                38,
                26,
                16
              ],
              "colorRange": {
                "name": "ColorBrewer Dark2-3",
                "type": "qualitative",
                "category": "ColorBrewer",
                "colors": [
                  "#1b9e77",
                  "#d95f02",
                  "#7570b3"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 10,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": False,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "bird",
              "type": "string"
            },
            "colorScale": "ordinal",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": {
              "name": "duration_h",
              "type": "real"
            },
            "radiusScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "stops": [
              {
                "name": "duration_h",
                "format": None
              },
              {
                "name": "length_m",
                "format": None
              },
              {
                "name": "bird",
                "format": None
              },
              {
                "name": "datetime",
                "format": None
              }
            ],
            "trajectories": [
              {
                "name": "index",
                "format": None
              },
              {
                "name": "altitude",
                "format": None
              },
              {
                "name": "date_time",
                "format": None
              },
              {
                "name": "device_info_serial",
                "format": None
              },
              {
                "name": "direction",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 33.37989411119247,
      "longitude": 25.985929059513722,
      "pitch": 0,
      "zoom": 3.249643007871771,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

In [2]:
import duckdb
from shapely.geometry import Point
import geopandas as gpd
from pyproj import CRS
# from keplergl import KeplerGl
import pandas as pd

In [3]:
duckdb_folder = '../duckdb'
data_folder = '../data'

con = duckdb.connect(duckdb_folder+'/kepler.duckdb')

In [4]:
data_raw = con.execute('SELECT * FROM "trips"').df()

In [5]:
data=data_raw[:20000]

In [6]:
data['geometry'] = [Point(long, lat) for long, lat in zip(data['posx'].to_list(), data['posy'].to_list())]
geodata = gpd.GeoDataFrame(data, crs = CRS.from_epsg('4326'))
geodata

/tmp/ipykernel_13674/2678564112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['geometry'] = [Point(long, lat) for long, lat in zip(data['posx'].to_list(), data['posy'].to_list())]


,tripid,vehicleid,posx,posy,t,geometry
0,1,1,4.457924,50.889049,2020-06-01 11:48:50.886000+02:00,POINT (4.45792 50.88905)
1,1,1,4.457894,50.889028,2020-06-01 11:48:52.386000+02:00,POINT (4.45789 50.88903)
2,1,1,4.457721,50.888909,2020-06-01 11:48:57.523414+02:00,POINT (4.45772 50.88891)
3,1,1,4.457928,50.888690,2020-06-01 11:49:05.623414+02:00,POINT (4.45793 50.88869)
4,1,1,4.457930,50.888689,2020-06-01 11:49:05.668849+02:00,POINT (4.45793 50.88869)
...,...,...,...,...,...,...
19995,10,1,4.410637,50.801712,2020-06-05 00:14:04.531535+02:00,POINT (4.41064 50.80171)
19996,10,1,4.410675,50.801728,2020-06-05 00:14:05.131535+02:00,POINT (4.41068 50.80173)
19997,10,1,4.410713,50.801743,2020-06-05 00:14:06.131535+02:00,POINT (4.41071 50.80174)
19998,10,1,4.411239,50.801961,2020-06-05 00:14:14.531535+02:00,POINT (4.41124 50.80196)


In [7]:
# Create timestamp with Pandas datetime. Inclue None as timezone
geodata['time'] = pd.to_datetime(geodata['t']).dt.tz_localize(None)
# Set timestamp as Index
geodata = geodata.set_index('time')
print(geodata.shape)
geodata.head()


(20000, 6)


,tripid,vehicleid,posx,posy,t,geometry
time,,,,,,
2020-06-01 11:48:50.886000,1,1,4.457924,50.889049,2020-06-01 11:48:50.886000+02:00,POINT (4.45792 50.88905)
2020-06-01 11:48:52.386000,1,1,4.457894,50.889028,2020-06-01 11:48:52.386000+02:00,POINT (4.45789 50.88903)
2020-06-01 11:48:57.523414,1,1,4.457721,50.888909,2020-06-01 11:48:57.523414+02:00,POINT (4.45772 50.88891)
2020-06-01 11:49:05.623414,1,1,4.457928,50.888690,2020-06-01 11:49:05.623414+02:00,POINT (4.45793 50.88869)
2020-06-01 11:49:05.668849,1,1,4.457930,50.888689,2020-06-01 11:49:05.668849+02:00,POINT (4.45793 50.88869)


In [8]:
import movingpandas as mpd
traj_collection = mpd.TrajectoryCollection(geodata, 'tripid')

/home/sakana/anaconda3/envs/mobil/lib/python3.10/site-packages/movingpandas/__init__.py:41: UserWarning: Missing optional dependencies. To use the trajectory smoother classes please install Stone Soup (see https://stonesoup.readthedocs.io/en/latest/#installation).
  warnings.warn(e.msg, UserWarning)


In [9]:
traj_collection

TrajectoryCollection with 10 trajectories

In [10]:
from datetime import datetime, timedelta
# Define parameters in Hours and Search radius in meters
Second = 1
Minutes = 1
Hours = 12
SearchRadio = 100
# stop detection
stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(seconds=1), max_diameter=SearchRadio)

In [11]:
stops

TrajectoryCollection with 1067 trajectories

In [12]:
stops_start = gpd.GeoDataFrame(columns = ['geometry'])
stops_start = stops_start.set_geometry('geometry')

In [13]:
stops_start['stop_id'] = [track.id for track in stops.trajectories]
stops_start= stops_start.set_index('stop_id')


In [14]:
for stoptrack in stops.trajectories:

    # add stop duration in hours
    stops_start.at[stoptrack.id,'duration_h'] =stoptrack.get_duration().total_seconds()/3600

    # add length
    stops_start.at[stoptrack.id, 'length_m']=stoptrack.get_length()

    # add bird name
    stops_start.at[stoptrack.id, 'tripid']=stoptrack.id.split('_')[0]

    # add datetime
    stops_start.at[stoptrack.id, 'datetime']= pd.to_datetime(stoptrack.id.split('_')[1]).tz_localize(None)

    # geometry with start point
    stops_start.at[stoptrack.id, 'geometry'] = stoptrack.get_start_location()
print(stops_start.shape)
stops_start.head()


(1067, 5)


,geometry,duration_h,length_m,tripid,datetime
stop_id,,,,,
1_2020-06-01 11:48:50.886000,POINT (4.45792 50.88905),0.008360,96.957224,1,2020-06-01 11:48:50.886000
1_2020-06-01 11:49:26.624003,POINT (4.45843 50.88818),0.009625,89.403874,1,2020-06-01 11:49:26.624003
1_2020-06-01 11:50:04.874237,POINT (4.45968 50.88763),0.009784,132.079704,1,2020-06-01 11:50:04.874237
1_2020-06-01 11:50:40.597012,POINT (4.46036 50.88704),0.004817,84.711824,1,2020-06-01 11:50:40.597012
1_2020-06-01 11:51:01.899785,POINT (4.4605 50.88603),0.007334,67.020078,1,2020-06-01 11:51:01.899785


In [15]:
# Reset indexes
stops_start = stops_start.reset_index(drop=True)
geodata= geodata.reset_index(drop=True)

In [16]:
from keplergl import KeplerGl

m = KeplerGl(height=600)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [17]:
# Add stop durations
m.add_data(stops_start, 'stops')
# Add gps records
m.add_data(geodata, 'trajectories')

In [18]:
m

KeplerGl(data={'stops': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

In [20]:
m.save_to_html(file_name='index.html', config=config, read_only=True)

Map saved to index.html!
